In [33]:
!pip install PyTDC
!pip install rdkit-pypi
!pip install mendeleev
# !pip install chemprop
!git clone https://github.com/chen-dawn/chemprop_edited.git
!pip install ./chemprop_edited
# !pip install numpy==1.16.1
# import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'chemprop_edited' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./chemprop_edited
  Preparing metadata (setup.py) ... done
  Created wheel for chemprop: filename=chemprop-0.0.0-py3-none-any.whl size=166223 sha256=6fa8e487631e6aecb88152f1f44848d726a2303d1143c96df1366280b336095a
  Stored in directory: /tmp/pip-ephem-wheel-cache-kcqeolwh/wheels/8e/8c/61/b42f70a49a192119b73bd84343042434d85c4b6aadfb9b0d21
Successfully built chemprop
  Attempting uninstall: chemprop
    Found existing installation: chemprop 0.0.0
    Uninstalling chemprop-0.0.0:
      Successfully unin

In [34]:
# !wget https://www.dropbox.com/s/nv520aj6osq52mz/features.zip
# ! unzip features.zip
# # !rm -rf chemprop

In [35]:
from tdc.single_pred import Tox
import chemprop
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors,Crippen
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
import sys
import torch 
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import mendeleev

from rdkit import RDLogger   
RDLogger.DisableLog('rdApp.*') # turn off RDKit warning message 

print(chemprop.__file__)

from google.colab import drive
# Mount google drive.
drive.mount('/content/drive')
mydrive = '/content/drive/MyDrive'

# Hack for the error
import numpy as np
# save np.load
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

!wget https://www.dropbox.com/s/eg12pajgvxgcf7n/SMARTS_functional_groups_upd2.txt

/usr/local/lib/python3.10/dist-packages/chemprop/__init__.py
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2023-05-12 05:22:39--  https://www.dropbox.com/s/eg12pajgvxgcf7n/SMARTS_functional_groups_upd2.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/eg12pajgvxgcf7n/SMARTS_functional_groups_upd2.txt [following]
--2023-05-12 05:22:40--  https://www.dropbox.com/s/raw/eg12pajgvxgcf7n/SMARTS_functional_groups_upd2.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf4e5560c2e805b0d611c0e3537.dl.dropboxusercontent.com/cd/0/inline/B76KB49Ex1vfR9khzeZwrEqmvHQKdI6R5P9RXRpIaROyI1wyEWZJ-VdBGMUkzK9fPffeScnFbMmdZnXr5hrEuE6PBF_OE7Q3eDUlcTNDitGsBgFONQRfs

In [36]:
# Get toxicity data and write to output. 
data = Tox(name = 'LD50_Zhu')
df = data.get_data()
# split = data.get_split(method = 'scaffold', seed = 42)
# split['train'].to_csv('./Tox_train.csv', index=False)
# split['valid'].to_csv('./Tox_val.csv', index=False)
# split['test'].to_csv('./Tox_test.csv', index=False)
df.to_csv('./Tox.csv', index=False)

Found local copy...
Loading...
Done!


In [37]:
# Create physical features 
atom_numbers = [*range(1,100)]
envs_list = np.array([0]+[mendeleev.element(i).en_pauling for i in atom_numbers])
radii_list = np.array([0]+[mendeleev.element(i).atomic_radius for i in atom_numbers])
polar_list = np.array([0]+[mendeleev.element(i).dipole_polarizability for i in atom_numbers])
elaff_list = np.array([0]+[mendeleev.element(i).electron_affinity for i in atom_numbers])
ionen_list = np.array([0]+[mendeleev.element(i).ionenergies[1] for i in atom_numbers])
for i in range(len(envs_list)):
  if envs_list[i] is None:
    envs_list[i] = 0

for i in range(len(radii_list)):
  if radii_list[i] is None:
    radii_list[i] = 0

for i in range(len(polar_list)):
  if polar_list[i] is None:
    polar_list[i] = 0

for i in range(len(elaff_list)):
  if elaff_list[i] is None:
    elaff_list[i] = 0

for i in range(len(ionen_list)):
  if ionen_list[i] is None:
    ionen_list[i] = 0

In [38]:
smiles = df['Drug']

radii = []
envs = []
polar = []
elaff = []
ionen = []
for SMILES in smiles:
    mol = Chem.MolFromSmiles(SMILES)
    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    radii.append([radii_list[i] for i in atoms])
    envs.append([envs_list[i] for i in atoms])
    polar.append([polar_list[i] for i in atoms])
    elaff.append([elaff_list[i] for i in atoms])
    ionen.append([ionen_list[i] for i in atoms])

ar = []
for r,e,p,el,io in zip(radii, envs, polar, elaff, ionen):
    ar.append(np.vstack([np.array(r), np.array(e), np.array(p), np.array(el), np.array(io)]).T)

# Create functional group features.
def get_indices_of_functional_groups(mol, smarts_pattern):
    """
    Returns the indices of atoms in a molecule that match a specified functional group pattern.
    If there are multiple matches, all the matches are returned as individual lists.

    :param mol: A molecule.
    :param smarts_pattern: A SMARTS pattern.
    :return: A nested list of atom indices, where each inner list corresponds to a single match.
    """
    
    pattern = Chem.MolFromSmarts(smarts_pattern)
    # find all matches to the pattern.
    occurance_list = []
    for i, match in enumerate(mol.GetSubstructMatches(pattern)):
        indices = []
        for j in match:
            mol.GetAtomWithIdx(j).SetProp('feature', str(i + 1))
            indices.append(j)
        occurance_list.append(indices)
    return occurance_list

# How to run the function above.
# print(get_indices_of_functional_groups(Chem.MolFromSmiles('CC(CS)C(=O)N1CCCC1C(=O)O'), '[CX3]=[OX1]'))

# Read in tab separated file of smarts patterns and functional group names
fg = pd.read_csv('./SMARTS_functional_groups_upd2.txt', sep='\t')

# Make a hash table of the reactivity. 
reactivity_dict = {}
for i, row in fg.iterrows():
    reactivity_dict[row['Description']] = row['REACTIVITY']
# Also make a hash table of the H_RELEVANT column.
h_relevant_dict = {}
for i, row in fg.iterrows():
    h_relevant_dict[row['Description']] = row['H_RELEVANT']

functional_groups_feature_all = []
reactivity_feature_all = []
h_relevant_feature_all = []

for SMILES in smiles:
    mol = Chem.MolFromSmiles(SMILES)
    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    num_atoms = len(atoms)
    # Create a list of the atom indices that match the SMARTS pattern and 0's for the rest.
    
    all_functional_groups = []
    all_reactivity = []
    all_h_relevant = []

    for i, row in fg.iterrows():
        SMARTS = row['SMARTS']
        functional_group = np.zeros(num_atoms)
        reactivity = np.zeros(num_atoms)
        h_relevant = np.zeros(num_atoms)

        # Get the indices of atoms that match the SMARTS pattern.
        indices = get_indices_of_functional_groups(mol, SMARTS)
        
        for j, index_list in enumerate(indices):
            for atom in index_list:
                functional_group[atom] = j + 1
                reactivity[atom] = reactivity_dict[row['Description']]
                h_relevant[atom] = h_relevant_dict[row['Description']]

        # if len(indices) > 0:
        #     print(row['Description'])
        #     print(functional_group)
        #     display(mol)

        all_functional_groups.append(functional_group)
        all_reactivity.append(reactivity)
        all_h_relevant.append(h_relevant)
    
    all_functional_groups = np.vstack(all_functional_groups).T
    functional_groups_feature_all.append(all_functional_groups)
    all_reactivity = np.vstack(all_reactivity).T
    reactivity_feature_all.append(all_reactivity)
    all_h_relevant = np.vstack(all_h_relevant).T
    h_relevant_feature_all.append(all_h_relevant)
# Save the data
np.savez('feature_physical_description.npz', *ar)
np.savez('feature_functional_group.npz', *functional_groups_feature_all)
np.savez('feature_h_relevant.npz', *h_relevant_feature_all)
np.savez('feature_reactivity.npz', *reactivity_feature_all)
# Concate physical group and functional group features for atoms. 
all = ar + functional_groups_feature_all + reactivity_feature_all + h_relevant_feature_all
np.savez('feature_all.npz', *all)

In [39]:
# Default only
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' 

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'constraints_path': None,
 'crossval_index_dir': None,
 'crossval_index_file

In [40]:
# Functional group only
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_functional_group.npz' \
 --overwrite_default_atom_features

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_functional_group.npz --overwrite_default_atom_features
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_functional_group.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 '

In [41]:
# Reactivity only 
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_reactivity.npz' \
 --overwrite_default_atom_features

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_reactivity.npz --overwrite_default_atom_features
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_reactivity.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_p

In [42]:
# H_relevant
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_h_relevant.npz' \
 --overwrite_default_atom_features

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_h_relevant.npz --overwrite_default_atom_features
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_h_relevant.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_p

In [43]:
# Physical description
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_physical_description.npz' \
 --overwrite_default_atom_features

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_physical_description.npz --overwrite_default_atom_features
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_physical_description.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': 

In [44]:
# Feature all
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_all.npz' \
 --overwrite_default_atom_features

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_all.npz --overwrite_default_atom_features
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_all.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 

In [45]:
# Feature all with default
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_all.npz'

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_all.npz
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_all.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_p

In [46]:
# Feature functional group with default
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_functional_group.npz'

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_functional_group.npz
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_functional_group.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_b

In [47]:
# Feature reactivity with default
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_reactivity.npz'

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_reactivity.npz
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_reactivity.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': Fal

In [48]:
# Feature H relevant with default
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_h_relevant.npz'

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_h_relevant.npz
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_h_relevant.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': Fal

In [49]:
# Feature physical description with default
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_physical_description.npz'

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_physical_description.npz
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_physical_description.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 

In [53]:
empty_set = []
for SMILES in smiles:
    mol = Chem.MolFromSmiles(SMILES)
    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    empty_set.append([0 for i in atoms])

ar = []
for e in empty_set:
    ar.append(np.vstack([np.array(e)]).T)


np.savez('feature_none.npz', *ar)

# Feature all
!chemprop_train --data_path 'Tox.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './feature_none.npz' \
 --overwrite_default_atom_features

Command line
python /usr/local/bin/chemprop_train --data_path Tox.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./feature_none.npz --overwrite_default_atom_features
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './feature_none.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,

In [14]:
!wget https://www.dropbox.com/s/463iose57ewwexk/PRISM_Clean_V2.csv
prism_df = pd.read_csv("./PRISM_Clean_V2.csv")

prism_df

--2023-05-12 04:54:35--  https://www.dropbox.com/s/463iose57ewwexk/PRISM_Clean_V2.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/463iose57ewwexk/PRISM_Clean_V2.csv [following]
--2023-05-12 04:54:35--  https://www.dropbox.com/s/raw/463iose57ewwexk/PRISM_Clean_V2.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc581dd066dab26b4e159d191513.dl.dropboxusercontent.com/cd/0/inline/B74_rVg93omPBdoPPg405W958hTIVHOJ5IvZ7hweiDzF_HTbKvbi2cZDiPnRan03NYTDmtP39xVCcOmDbKrl0kZ_AXIjrfWlYhsCjOjtig2-xmOBQBy_O6DwewJtlhyXKVufx4NR3ji0XoZpw3C_2EDr6fXPy7MhV3f_YblRvpdong/file# [following]
--2023-05-12 04:54:38--  https://uc581dd066dab26b4e159d191513.dl.dropboxusercontent.com/cd/0/inline/B74_rVg93omPBdoPPg405W958hTIVHOJ5IvZ7hweiDzF_HTbKvbi2cZDiPn

,Drug,Y
0,CCOC(=O)C1CN(Cc2ccccc2)C(=O)C1=O,-0.262909
1,COc1ccc2Sc3ccccc3N(C[C@H](C)CN(C)C)c2c1,0.562749
2,CCCN(CCC)C1CCc2cccc(O)c2C1,0.459108
3,CCOc1ccccc1OCC1CNCCO1,0.497116
4,O=C(NCCCN1CCCCCC1)C(C1CCCCC1)c1ccccc1,0.101745
...,...,...
4172,CN1c2ccccc2C(NCCCCCCC(O)=O)c2ccc(Cl)cc2S1(=O)=O,0.343799
4173,COc1ccc(cc1O[C@H]1C[C@@H]2CC[C@H]1C2)C1CNC(=O)NC1,0.439155
4174,COc1cccc(c1F)-c1c(C)n(Cc2c(F)cccc2C(F)(F)F)c(=...,-0.019961
4175,CC(C)(C)NCC(O)c1cc(Cl)c(N)c(c1)C(F)(F)F,0.247897


In [15]:
smiles = prism_df['Drug']
radii = []
envs = []
polar = []
elaff = []
ionen = []
for SMILES in smiles:
    mol = Chem.MolFromSmiles(SMILES)
    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    radii.append([radii_list[i] for i in atoms])
    envs.append([envs_list[i] for i in atoms])
    polar.append([polar_list[i] for i in atoms])
    elaff.append([elaff_list[i] for i in atoms])
    ionen.append([ionen_list[i] for i in atoms])

ar = []
for r,e,p,el,io in zip(radii, envs, polar, elaff, ionen):
    ar.append(np.vstack([np.array(r), np.array(e), np.array(p), np.array(el), np.array(io)]).T)

# Create functional group features.
def get_indices_of_functional_groups(mol, smarts_pattern):
    """
    Returns the indices of atoms in a molecule that match a specified functional group pattern.
    If there are multiple matches, all the matches are returned as individual lists.

    :param mol: A molecule.
    :param smarts_pattern: A SMARTS pattern.
    :return: A nested list of atom indices, where each inner list corresponds to a single match.
    """
    
    pattern = Chem.MolFromSmarts(smarts_pattern)
    # find all matches to the pattern.
    occurance_list = []
    for i, match in enumerate(mol.GetSubstructMatches(pattern)):
        indices = []
        for j in match:
            mol.GetAtomWithIdx(j).SetProp('feature', str(i + 1))
            indices.append(j)
        occurance_list.append(indices)
    return occurance_list

# How to run the function above.
# print(get_indices_of_functional_groups(Chem.MolFromSmiles('CC(CS)C(=O)N1CCCC1C(=O)O'), '[CX3]=[OX1]'))

# Read in tab separated file of smarts patterns and functional group names
fg = pd.read_csv('./SMARTS_functional_groups_upd2.txt', sep='\t')

# Make a hash table of the reactivity. 
reactivity_dict = {}
for i, row in fg.iterrows():
    reactivity_dict[row['Description']] = row['REACTIVITY']
# Also make a hash table of the H_RELEVANT column.
h_relevant_dict = {}
for i, row in fg.iterrows():
    h_relevant_dict[row['Description']] = row['H_RELEVANT']

functional_groups_feature_all = []
reactivity_feature_all = []
h_relevant_feature_all = []

for SMILES in smiles:
    mol = Chem.MolFromSmiles(SMILES)
    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    num_atoms = len(atoms)
    # Create a list of the atom indices that match the SMARTS pattern and 0's for the rest.
    
    all_functional_groups = []
    all_reactivity = []
    all_h_relevant = []

    for i, row in fg.iterrows():
        SMARTS = row['SMARTS']
        functional_group = np.zeros(num_atoms)
        reactivity = np.zeros(num_atoms)
        h_relevant = np.zeros(num_atoms)

        # Get the indices of atoms that match the SMARTS pattern.
        indices = get_indices_of_functional_groups(mol, SMARTS)
        
        for j, index_list in enumerate(indices):
            for atom in index_list:
                functional_group[atom] = j + 1
                reactivity[atom] = reactivity_dict[row['Description']]
                h_relevant[atom] = h_relevant_dict[row['Description']]

        # if len(indices) > 0:
        #     print(row['Description'])
        #     print(functional_group)
        #     display(mol)

        all_functional_groups.append(functional_group)
        all_reactivity.append(reactivity)
        all_h_relevant.append(h_relevant)
    
    all_functional_groups = np.vstack(all_functional_groups).T
    functional_groups_feature_all.append(all_functional_groups)
    all_reactivity = np.vstack(all_reactivity).T
    reactivity_feature_all.append(all_reactivity)
    all_h_relevant = np.vstack(all_h_relevant).T
    h_relevant_feature_all.append(all_h_relevant)
# Save the data
np.savez('prism_feature_physical_description.npz', *ar)
np.savez('prism_feature_functional_group.npz', *functional_groups_feature_all)
np.savez('prism_feature_h_relevant.npz', *h_relevant_feature_all)
np.savez('prism_feature_reactivity.npz', *reactivity_feature_all)
# Concate physical group and functional group features for atoms. 
all = ar + functional_groups_feature_all + reactivity_feature_all + h_relevant_feature_all
np.savez('prism_feature_all.npz', *all)

In [16]:
# Physical descriptions only 
!chemprop_train --data_path 'PRISM_Clean_V2.csv' --dataset_type 'regression'\
 --smiles_column 'Drug' --target_columns 'Y' --save_dir './' \
 --atom_descriptors feature --atom_descriptors_path './prism_feature_functional_group.npz' \
 --split_type scaffold_balanced --overwrite_default_atom_features

Command line
python /usr/local/bin/chemprop_train --data_path PRISM_Clean_V2.csv --dataset_type regression --smiles_column Drug --target_columns Y --save_dir ./ --atom_descriptors feature --atom_descriptors_path ./prism_feature_functional_group.npz --split_type scaffold_balanced --overwrite_default_atom_features
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': 'feature',
 'atom_descriptors_path': './prism_feature_functional_group.npz',
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
